In [52]:
!wget https://raw.githubusercontent.com/karpathy/makemore/master/names.txt

--2023-10-13 13:33:50--  https://raw.githubusercontent.com/karpathy/makemore/master/names.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228145 (223K) [text/plain]
Saving to: ‘names.txt.2’

names.txt.2         100%[===================>] 222,80K  --.-KB/s    in 0,06s   

2023-10-13 13:33:50 (3,95 MB/s) - ‘names.txt.2’ saved [228145/228145]



In [1]:
with open("names.txt", "r") as f:
    words = f.read().splitlines()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [2]:
vocab = set("".join(words + ["."]))
len(vocab)

27

In [3]:
itoch = {i:ch for i, ch in enumerate(set("".join(words + ["."])))}
chtoi = {ch:i for i, ch in itoch.items()}
chtoi

{'n': 0,
 'u': 1,
 's': 2,
 'f': 3,
 '.': 4,
 'e': 5,
 'm': 6,
 'o': 7,
 'z': 8,
 't': 9,
 'g': 10,
 'q': 11,
 'l': 12,
 'p': 13,
 'k': 14,
 'a': 15,
 'v': 16,
 'y': 17,
 'r': 18,
 'j': 19,
 'w': 20,
 'd': 21,
 'x': 22,
 'b': 23,
 'h': 24,
 'i': 25,
 'c': 26}

In [4]:
import torch

def make_data(words, block_size=3):
    Xs, Ys = [], []
    for word in words:
        block = [chtoi["."]] * block_size
        for letter in word + ".":
            letter_int = chtoi[letter]
            Xs.append(block)
            Ys.append(letter_int)
            block = block[1:] + [letter_int]
    return torch.tensor(Xs), torch.tensor(Ys)

In [176]:
import random

random.seed(7)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = make_data(words[:n1])
Xval, Yval = make_data(words[n1:n2])
Xtest, Ytest = make_data(words[n2:])


In [177]:
print(Xtr)
print(Ytr)

tensor([[ 4,  4,  4],
        [ 4,  4,  8],
        [ 4,  8, 15],
        ...,
        [21,  5,  2],
        [ 5,  2,  9],
        [ 2,  9,  7]])
tensor([ 8, 15, 25,  ...,  9,  7,  4])


In [8]:
# Xtr = Xtr[:1]
# Ytr = Ytr[:1]

In [9]:
# two demensional space for every world 
C = torch.randn((27,2))

In [10]:
print(C[Xtr[0]])
# ---> need to predict
print(Ytr[0])

tensor([[-0.4127,  1.3877],
        [-0.4127,  1.3877],
        [-0.4127,  1.3877]])
tensor(14)


In [113]:
import torch.nn.functional as F
assert all(F.one_hot(torch.tensor(4), num_classes=27).float() @ C == C[4])

In [11]:
# forward pass
C[Xtr].shape

torch.Size([1, 3, 2])

In [26]:
emb = C[Xtr]
emb.shape

torch.Size([1, 3, 2])

In [29]:
W = torch.randn((6, 100))
b = torch.randn((100,))
emb @ W + b

RuntimeError: mat1 and mat2 shapes cannot be multiplied (3x2 and 6x100)

In [81]:
emb

tensor([[[-0.4127,  1.3877],
         [-0.4127,  1.3877],
         [-0.4127,  1.3877]]])

In [82]:
# chcemy z emb o wymiaryach 1,3,2 zmienić na wymiary 1, 6 
torch.cat((emb[:,0,:], emb[:,1,:], emb[:,2,:]), dim=1)


tensor([[-0.4127,  1.3877, -0.4127,  1.3877, -0.4127,  1.3877]])

In [83]:
torch.cat((emb[:,0,:], emb[:,1,:], emb[:,2,:]), dim=1).shape

torch.Size([1, 6])

In [84]:
torch.cat((emb[:,0], emb[:,1], emb[:,2]), dim=1) @ W + b
# dla każdego z 27 liter prawdpodobienstwa przed normalizacją 

tensor([[ 4.6216,  0.3482, -0.0828, -1.3032, -0.1063,  0.1853, -5.0332, -2.7573,
          2.0218,  0.7074,  3.8027, -0.2592, -0.5935,  3.7609,  0.3063, -1.7958,
          0.6337, -3.4049, -2.2718,  3.5253, -3.8619,  1.9599,  0.6750, -1.6466,
          3.7459, -0.3117,  1.4384,  0.5669,  6.7886,  0.8523,  4.2219,  0.3101,
          2.9659, -0.2958, -0.9242,  0.4876,  3.6477, -1.2454,  0.0566, -2.1039,
         -4.3322,  0.2648, -1.4948,  5.9703, -1.3876,  2.1602, -0.9177,  3.2663,
          1.2937, -0.5446, -1.1439,  0.5079,  0.2678,  1.9708,  0.5263, -0.4662,
         -2.8326, -3.6063, -0.7960,  0.1086, -0.7372,  1.0902, -0.1860, -0.5168,
          0.4198,  0.4554, -3.8034,  1.6933, -2.2972, -1.7961,  2.0506, -2.0018,
          2.6003,  1.0769,  0.3389, -3.0098, -4.1756,  5.7027, -1.7722,  0.6083,
          0.8145,  4.1647,  0.7487, -1.9342, -0.7743, -3.1810, -1.3212, -2.0557,
          0.5819, -1.3590,  2.6757, -7.0366, -2.4989, -4.6825, -1.6527,  0.0792,
         -1.6315,  2.4325, -

In [85]:
torch.unbind(emb, 1)

(tensor([[-0.4127,  1.3877]]),
 tensor([[-0.4127,  1.3877]]),
 tensor([[-0.4127,  1.3877]]))

In [86]:
torch.cat(torch.unbind(emb, 1), 1)
# same thing 

tensor([[-0.4127,  1.3877, -0.4127,  1.3877, -0.4127,  1.3877]])

In [87]:
emb.storage()

 -0.41270169615745544
 1.3876614570617676
 -0.41270169615745544
 1.3876614570617676
 -0.41270169615745544
 1.3876614570617676
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [88]:
# the best, becouse its only change .storage 
# to make vie work 1 * 6 == len(emb)
emb.view(-1,6)

tensor([[-0.4127,  1.3877, -0.4127,  1.3877, -0.4127,  1.3877]])

In [91]:
W = torch.randn((6, 100))
b = torch.randn((100,))

h = torch.tanh(emb.view(-1,6) @ W + b)
h.shape

torch.Size([1, 100])

In [93]:
W2 = torch.randn((100, 27))
b2 = torch.randn((27,))

logits = h @ W2 + b2

In [97]:
counts = logits.exp()
p = counts / counts.sum(1, keepdim=True)
p

tensor([[8.4335e-08, 7.3455e-05, 2.2849e-05, 7.3862e-10, 8.4030e-02, 7.9087e-02,
         6.8150e-06, 3.8965e-09, 4.0158e-05, 6.7844e-06, 9.2947e-07, 8.3669e-01,
         8.7415e-10, 9.2508e-09, 2.1681e-09, 1.1023e-11, 2.2570e-08, 4.9096e-15,
         1.9839e-05, 5.7018e-10, 1.6492e-19, 5.6202e-08, 2.2967e-06, 1.4468e-05,
         6.4654e-09, 1.8073e-09, 6.8012e-07]])

In [108]:
import torch.nn.functional as F
p = F.softmax(logits, dim=1)
p

tensor([[8.4335e-08, 7.3455e-05, 2.2849e-05, 7.3862e-10, 8.4030e-02, 7.9087e-02,
         6.8150e-06, 3.8965e-09, 4.0158e-05, 6.7844e-06, 9.2947e-07, 8.3669e-01,
         8.7415e-10, 9.2508e-09, 2.1681e-09, 1.1023e-11, 2.2570e-08, 4.9096e-15,
         1.9839e-05, 5.7018e-10, 1.6492e-19, 5.6202e-08, 2.2967e-06, 1.4468e-05,
         6.4654e-09, 1.8072e-09, 6.8012e-07]])

In [115]:
loss = -p[torch.arange(Ytr.shape[0]), Ytr].log().mean()
loss

tensor(19.9494)

In [116]:
# F.cross_entropy w PyTorch to funkcja, która oblicza negative log-likelihood 
# loss w połączeniu z log_softmax. Oznacza to, że nie musisz stosować osobno softmax i log_softmax
loss = F.cross_entropy(logits, Ytr)
loss


tensor(19.9494)

# sum up

In [174]:
C = torch.randn((27,2))

W = torch.randn((6, 100))
b = torch.randn(100)

W2 = torch.randn((100, 27))
b2 = torch.randn(27)

parms = [C, W, b, W2, b2]
for parm in parms:
    parm.requires_grad = True

In [178]:
for _ in range(100):
    # forward
    h = torch.tanh(C[Xtr].reshape(-1, 6) @ W  + b)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr)
    print(f'{loss.item()=}')

    # backward 
    loss.backward()
    for parm in parms:
        parm.data += -0.1 * parm.grad
    


loss.item()=329.15771484375
loss.item()=225.9630889892578
loss.item()=192.48878479003906
loss.item()=157.08538818359375
loss.item()=88.80526733398438
loss.item()=58.40372848510742
loss.item()=55.04838562011719
loss.item()=43.11375427246094
loss.item()=52.42787551879883
loss.item()=53.36442565917969
loss.item()=46.63205337524414
loss.item()=44.758872985839844
loss.item()=60.14103317260742
loss.item()=58.75661849975586
loss.item()=58.10542297363281
loss.item()=49.84859085083008
loss.item()=51.75644302368164
loss.item()=42.50431823730469
loss.item()=35.068450927734375
loss.item()=30.58955955505371
loss.item()=29.65923500061035
loss.item()=29.001667022705078
loss.item()=27.895158767700195
loss.item()=28.654075622558594
loss.item()=29.265350341796875
loss.item()=27.6474609375
loss.item()=27.754106521606445
loss.item()=26.88087272644043
loss.item()=25.506668090820312
loss.item()=23.8644962310791
loss.item()=23.3826961517334
loss.item()=20.55409812927246
loss.item()=18.59749412536621
loss.ite